In [2]:
import os
import re
from os.path import exists
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import codecs
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import glob
import scipy.sparse
from sklearn.decomposition import TruncatedSVD
import tempfile
import pickle



In [3]:
directory = 'parsed_files'

all_files = glob.glob(f"{directory}/*")
tfidf_vec = TfidfVectorizer(input ='filename')

matrix = tfidf_vec.fit_transform(all_files)
# matrix_file_name = "tfidf_matrix"
# scipy.sparse.save_npz(matrix_file_name, matrix, compressed=True)

In [11]:
tfidf_matrix = scipy.sparse.load_npz("tfidf_matrix.npz")
# tfidf_matrix = tfidf_matrix.T
print(tfidf_matrix.shape)

a_file = open("union.pkl", "wb")
pickle.dump(tfidf_vec.vocabulary_, a_file)
a_file.close()



(45361, 1182713)


In [15]:
a_file = open("union.pkl", "rb")
vocab = pickle.load(a_file)
a_file.close()


In [99]:
tfidf_matrix = tfidf_matrix.T
print(tfidf_matrix.shape)


(1182713, 45361)


In [125]:
import numpy as np
from sklearn.preprocessing import normalize
# inp = input("enter keywords")

inp = 'american football team'
stemmer = PorterStemmer()

stemmed_words = [stemmer.stem(word) for word in inp.split()]
print(stemmed_words)


x = np.zeros(shape=tfidf_matrix.shape[0])

for word in stemmed_words:
    if vocab[word]:
        x[vocab[word]]+= 1    


def vector_correlation(vec_q, vec_d):
    #vec q is like a row
    norm_q = scipy.sparse.linalg.norm(vec_q)
    norm_d = scipy.sparse.linalg.norm(vec_d)

    return (vec_q @ vec_d) / (norm_q * norm_d)

sparse_x = scipy.sparse.csr_matrix(x)


['american', 'footbal', 'team']


In [34]:

# print( appention)
# print(all_files[appention])
res = []
for i in range(tfidf_matrix.shape[1]-int(35e3)):
    x = vector_correlation(sparse_x, tfidf_matrix.getcol(i)).data
    if len(x) > 0:
        res.append((i,x))

        
res.sort(key=lambda x: x[1], reverse=True)

for i in res[:15]:
    print(all_files[i[0]])


parsed_files\11047@Eastern_Pacific_Ocean.txt
parsed_files\18458@Indian_Ocean.txt
parsed_files\17356@History_of_the_Union_Pacific_Railroad.txt
parsed_files\11121@East_Pacific_Rise.txt
parsed_files\1594@Atlantic.txt
parsed_files\1596@Atlantic_Basin.txt
parsed_files\1602@Atlantic_Ocean.txt
parsed_files\18041@Hydrographer.txt
parsed_files\18043@Hydrography.txt
parsed_files\14787@Geological_history_of_Earth.txt
parsed_files\10305@Division_No._1,_Saskatchewan.txt
parsed_files\18459@Indian_Ocean_(band).txt
parsed_files\19253@Isthmus_of_Panama.txt
parsed_files\15202@Global_surface_temperature.txt
parsed_files\1619@Atoll.txt


In [105]:
print(matrix.shape)
svd = TruncatedSVD(n_components=100).fit(matrix)
svd_matrix = svd.transform(matrix)
svd_components = svd.components_

# self.save_svd(svd_matrix, svd_components)

(45361, 1182713)


In [60]:
matrix_normalized = normalize(tfidf_matrix, axis=0)
q = normalize(sparse_x)
# print(q @ matrix_normalized)
res = q @ svd_matrix
# res.sort(key = lambda x)
b = []
for j, i in enumerate(res.data):
    b.append((j, i))
# print(res.data)
b.sort(key = lambda x: x[1], reverse= True)
# print(b)
for i in range(15):
    print(all_files[b[i][0]])

NotImplementedError: multi-dimensional sub-views are not implemented

In [126]:
# q = scipy.sparse.lil_matrix((len(vocab), 1))
# term_ids = list(set([vocab[term] for term in set(stemmer.stem(inp)) if term in vocab]))

def vector_correlation(vec_q, A_k, vec_d):
    #vec q is like a row
    print(A_k.shape)
    print(vec_d.shape)
    norm_q = scipy.sparse.linalg.norm(vec_q)
    norm_d = np.linalg.norm(A_k @ vec_d.T)

    return (vec_q @ A_k @ vec_d) / (norm_q * norm_d)

# res = []
# appention = -1
# for i in range(svd_matrix.shape[1]):
#     x = vector_correlation(sparse_x,svd_matrix, svd_matrix[:,i])
#     if len(x) > 0:
#         res.append((i,x[0]))
    

svd_q = svd_components @ x
svd_c = svd_matrix @ svd_q

correlations = {document_id: svd_c[document_id] for document_id in range(len(all_files))}
import operator
results = sorted(correlations.items(), key=operator.itemgetter(1), reverse=True)

number_of_results = 10
for result in results[:number_of_results]:
    print(all_files[result[0]])
    # print(result[0])

parsed_files\16027@Gridiron_football.txt
parsed_files\74@American_football_in_the_United_States.txt
parsed_files\1034@Arena_football.txt
parsed_files\18516@Indoor_American_football.txt
parsed_files\67@American_football.txt
parsed_files\13756@Football_League_(disambiguation).txt
parsed_files\1469@Association_football.txt
parsed_files\27966@National_Football_League_(disambiguation).txt
parsed_files\26828@Modern_history_of_American_football.txt
parsed_files\13747@Football.txt


In [83]:
res.sort(key=lambda x: x[1], reverse=True)

for i in res[:15]:
    print(all_files[i[0]])

parsed_files\10073@Dindigul_District.txt
parsed_files\10047@Digitizer.txt
parsed_files\10002@Differential_geometry.txt
parsed_files\1003@Area_code_574.txt
parsed_files\10021@Digital_Data_Storage.txt
parsed_files\10059@Dilution_gene.txt
parsed_files\10017@Digital_content.txt
parsed_files\10089@Diplomacy.txt
parsed_files\10060@Dilwale_Dulhania_Le_Jayenge.txt
parsed_files\10051@Digne-les-Bains_(disambiguation).txt
parsed_files\10046@Digital_transformation.txt
parsed_files\10058@Dilly_Knox.txt
parsed_files\10076@Diocese#Archdiocese.txt
parsed_files\10040@Digital_rights_management.txt
parsed_files\10032@Digital_physics.txt


In [ ]:
from http.server import HTTPServer, BaseHTTPRequestHandler, SimpleHTTPRequestHandler

class Serv(BaseHTTPRequestHandler):
    
    def _set_response(self):
        print('set response')
        self.send_response(200)
        self.set_header("Access-Control-Allow-Origin", "*")

        self.send_header('Content-type', 'text/html')

        self.end_headers()

    def do_GET(self):
        print('do get')

        self._set_response()
        # self.wfile.write("GET request for {}".format(self.path).encode('utf-8'))

    def do_POST(self):
        print('do post')
        self._set_response()




httpd = HTTPServer(('localhost',8042),Serv)
httpd.serve_forever()
httpd.server_close()